In [1]:
from gnn import GCNNode, GCNGraph
import gnn
import torch
import helper_funcs
from torch_geometric.datasets import TUDataset
from torch_geometric.datasets import Planetoid
import torch.nn.functional as F
from torch_geometric.loader import DataLoader

In [2]:
# import 3 datasets
imdb_dataset = TUDataset(root='/tmp/IMDB-BINARY', name='IMDB-BINARY')
cora_dataset = Planetoid(root='/tmp/Cora', name='Cora')
enzyme_dataset = TUDataset(root='/tmp/ENZYMES', name='ENZYMES')

In [3]:
# global variables
EPOCHS = 200

# Node classifcation: CORA

In [4]:
# init model and optimizer
model = GCNNode(cora_dataset)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

# train model
for epoch in range(EPOCHS):
    model = gnn.GCNNode_train(model, cora_dataset, optimizer)

# calculate accuracy
model.eval()
pred = model(cora_dataset).argmax(dim=1)
correct = (pred[cora_dataset.test_mask] == cora_dataset.y[cora_dataset.test_mask]).sum()
acc = int(correct) / int(cora_dataset.test_mask.sum())
print(f'Accuracy: {acc:.4f}')

Accuracy: 0.8100


# Graph classifcation: IMDB

In [5]:
# set up dataset
train_loader, test_loader = helper_funcs.preprocess_data(imdb_dataset, onehot=True, batch_size=64)

In [6]:
# train model
model = GCNGraph(hidden_channels=64, dataset = imdb_dataset)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = torch.nn.CrossEntropyLoss()

for epoch in range(EPOCHS):
    gnn.GCNGraph_train(model, criterion, optimizer, train_loader)
    test_acc = gnn.GCNGraph_test(model, test_loader)
print(f'Test Acc: {test_acc:.4f}')

Test Acc: 0.8875


# Graph classification: ENZYME

In [7]:
# set up dataset
train_loader, test_loader = helper_funcs.preprocess_data(enzyme_dataset, onehot=False, batch_size=64)

In [8]:
# train model
model = GCNGraph(hidden_channels= 64, dataset = enzyme_dataset)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = torch.nn.CrossEntropyLoss()

for epoch in range(EPOCHS*10):
    gnn.GCNGraph_train(model, criterion, optimizer, train_loader)
    test_acc = gnn.GCNGraph_test(model, test_loader)
print(f'Test Acc: {test_acc:.4f}')

Test Acc: 0.5583
